In [4]:
import tkinter as tk
from tkinter import messagebox
from tkcalendar import DateEntry
import hashlib
import os
from datetime import datetime

from cryptography.hazmat.primitives import serialization, hashes  # Add hashes module
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.backends import default_backend

 
def generate_hash_Value():
  
    name = entry_name.get().strip().lower() 
    dob_date = dob_calendar.get_date() 
    dob = dob_date.strftime("%Y-%m-%d") 
    gender = gender_var.get()   
    nationality = entry_nationality.get().strip()
    father_name = entry_father_name.get().strip()
    mother_name = entry_mother_name.get().strip()

     
    if not all([name, dob, gender, nationality, father_name, mother_name]):
        messagebox.showerror("Error", "All fields are required!")
        return

    
    normalized_name = name.lower()

    
    certificate_text = f"Name: {name}\nDate of Birth: {dob}\nGender: {gender}\nNationality: {nationality}\nFather's Name: {father_name}\nMother's Name: {mother_name}"

     
    hashed_certificate = hashlib.sha256(certificate_text.encode()).hexdigest()

    
    hash_label.config(text="Hash Value: " + hashed_certificate)

    # Print hash value to console
    print("Hash Value:", hashed_certificate)

     
    file_path = "certificates.txt"

    
    if name_dob_exists(normalized_name, dob, file_path):
        messagebox.showerror("Error", "Certificate with same details already issued!")
        return

    
    with open(file_path, "a") as file:
        file.write(f"{normalized_name},{dob},{gender},{nationality},{father_name},{mother_name},{hashed_certificate}\n")

   
    certificate_file_path = f"{name}_certificate.txt"
    with open(certificate_file_path, "w") as cert_file:
        cert_file.write(certificate_text)

  
    private_key = load_private_key()

   
    
    encrypted_hash = private_key.sign(
        hashed_certificate.encode(),
        padding.PSS(
            mgf=padding.MGF1(hashes.SHA256()),
            salt_length=padding.PSS.MAX_LENGTH
        ),
        hashes.SHA256()
    )

   
    encrypted_hash_file_path = f"{name}_encrypted_hash.txt"
    with open(encrypted_hash_file_path, "wb") as hash_file:
        hash_file.write(encrypted_hash)

    messagebox.showinfo("Certificate Issued", "Digital certificate issued successfully!\nCertificate details saved in a text file.")


   
    

def name_dob_exists(name, dob, file_path):
    if os.path.exists(file_path):
        with open(file_path, "r") as file:
            for line in file:
                parts = line.strip().split(",")
                if len(parts) == 7:  
                    stored_name = parts[0].lower()  
                    stored_dob = parts[1]
                    if name == stored_name and dob == stored_dob:
                        return True
    return False

def load_private_key():
    with open("private_key.pem", "rb") as key_file:
        private_key = serialization.load_pem_private_key(
            key_file.read(),
            password=None,
            backend=default_backend()
        )
    return private_key


root = tk.Tk()
root.title("Digital Certificate Management")


label_name = tk.Label(root, text="Name:")
label_name.grid(row=0, column=0)
entry_name = tk.Entry(root)
entry_name.grid(row=0, column=1)

label_dob = tk.Label(root, text="Date of Birth:")
label_dob.grid(row=1, column=0)
dob_calendar = DateEntry(root)  
dob_calendar.grid(row=1, column=1)

label_gender = tk.Label(root, text="Gender:")
label_gender.grid(row=2, column=0)
gender_var = tk.StringVar(root)
gender_var.set("Male") 
gender_dropdown = tk.OptionMenu(root, gender_var, "Male", "Female")
gender_dropdown.grid(row=2, column=1)

label_nationality = tk.Label(root, text="Nationality:")
label_nationality.grid(row=3, column=0)
entry_nationality = tk.Entry(root)
entry_nationality.grid(row=3, column=1)

label_father_name = tk.Label(root, text="Father's Name:")
label_father_name.grid(row=4, column=0)
entry_father_name = tk.Entry(root)
entry_father_name.grid(row=4, column=1)

label_mother_name = tk.Label(root, text="Mother's Name:")
label_mother_name.grid(row=5, column=0)
entry_mother_name = tk.Entry(root)
entry_mother_name.grid(row=5, column=1)

hash_label = tk.Label(root, text="Hash Value:")
hash_label.grid(row=6, column=0, columnspan=2)

button_generate_hash_Value = tk.Button(root, text="Generate hash value", command=generate_hash_Value)
button_generate_hash_Value.grid(row=7, column=0, columnspan=2)

root.mainloop()

Hash Value: 32b826064595f85f9dd07523dfcafa92751377974ac02d4a1c8db3ff56e40975
